# 概要

Character-LevelのCNNでWAFを作る。  
論文の内容をKerasで実装してみる。  
原著論文はこちら http://iyatomi-lab.info/sites/default/files/user/CSPA2018%20Proceedings_ito.pdf 


## データの準備

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Embedding
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K

In [4]:
%cd /content/drive/MyDrive/WAffle/
!ls -al

/content/drive/MyDrive/WAffle
total 19
-rw------- 1 root root  151 Dec 18 14:37 2020-12-18.gslides
-rw------- 1 root root 9706 Dec 28 13:56 CharacterLevel-CNN
drwx------ 4 root root 4096 Dec 27 18:45 Dataset
drwx------ 2 root root 4096 Dec 27 13:42 KerasPractice
-rw------- 1 root root  151 Oct 26 12:09 WAF_Overview_補足資料.gslides


In [11]:
df = pd.read_csv('Dataset/cisc_database/csic_database.csv')
df.head(20)

,Unnamed: 0,Method,User-Agent,Pragma,Cache-Control,Accept,Accept-encoding,Accept-charset,language,host,cookie,content-type,connection,lenght,content,classification,URL
0,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=1F767F17239C9B670A39E9B10C3825F4,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/index.jsp HTTP/1.1
1,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=81761ACA043B0E6014CA42A4BCD06AB5,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/anadir.j...
2,Normal,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=933185092E0B668B90676E0A2B0767AF,application/x-www-form-urlencoded,Connection: close,Content-Length: 68,id=3&nombre=Vino+Rioja&precio=100&cantidad=55&...,0,http://localhost:8080/tienda1/publico/anadir.j...
3,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=8FA18BA82C5336D03D3A8AFA3E68CBB0,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/autentic...
4,Normal,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=7104E6C68A6BCF1423DAE990CE49FEE2,application/x-www-form-urlencoded,Connection: close,Content-Length: 63,modo=entrar&login=choong&pwd=d1se3ci%F3n&remem...,0,http://localhost:8080/tienda1/publico/autentic...
5,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=5254A7A7C7F0E74D699E6A9C9487795A,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/caracter...
6,Normal,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=417DBC69B2F909CCD574CFC69F56779F,application/x-www-form-urlencoded,Connection: close,Content-Length: 4,id=2,0,http://localhost:8080/tienda1/publico/caracter...
7,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=42B53DB3F82E67BBB24E57E9CDC8E008,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/carrito....
8,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=A651CE9798E1B71FCA9D9B2159432A22,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/entrar.j...
9,Normal,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=EE6A06EE8174FC2B05432AA5A0FCE998,application/x-www-form-urlencoded,Connection: close,Content-Length: 33,errorMsg=Credenciales+incorrectas,0,http://localhost:8080/tienda1/publico/entrar.j...


In [13]:
x_train, y_train = train_test_split(df, train_size=0.8, test_size=0.2, random_state=42)

In [14]:
x_train, x_test = train_test_split(x_train, train_size=0.8, test_size=0.2, random_state=42)
y_train, y_test = train_test_split(y_train, train_size=0.8, test_size=0.2, random_state=42)

In [15]:
print('x_train shape', x_train.shape)
print('y_train shape', y_train.shape)
print('x_test shape', x_test.shape)
print('y_test shape', y_test.shape)

x_train shape (39081, 17)
y_train shape (9770, 17)
x_test shape (9771, 17)
y_test shape (2443, 17)


## アーキテクチャ

1. Input(1000 characters)
2. Embedding(128demensions)
3. Conv(K size, 64) -> RELU
4. Max Pooling(K size)
5. Conv(K size, 64) 
6. Max Pooling(size of Conv 5's output)
7. concat output & reshape into 64 length vector
8. FCN(64 units)
9. Batch normalize
10. Dropout 0.5
11. FCN(sigmoid, 1 unit)

In [19]:
#Input Layer
inputs = Input(shape=(input_size,), name='character_input', dtype='int')

NameError: ignored

In [ ]:
#Embedding Layer
x = Embedding(alphabet_size +1, embedding_size, input_length=input_size)(inputs)

In [ ]:
#Convolution Layers
